In [1]:
from goatools import obo_parser
from zipfile import ZipFile

import os
import gzip
import graco
import shutil
import requests
import numpy as np
import pandas as pd
import networkx as nx
import Bio.UniProt.GOA as GOA

In [2]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

DATA_DIRECTORY = "/media/clusterduck123/joe/data/"
RAW_DATA_DIRECTORY = f"{DATA_DIRECTORY}/raw-data"
YEAST_DIRECTORY = f"{DATA_DIRECTORY}/processed-data/yeast"
NETWORK_DIRECTORY = f"{YEAST_DIRECTORY}/networks"
ANNOTATION_DIRECTORY = f"{YEAST_DIRECTORY}/annotations"

# Downloads

## BioGRID

In [3]:
if not os.path.exists(RAW_DATA_DIRECTORY):
     os.makedirs(directory)

VERSION = "3.5.178"
BioGRID_FILENAME = f"BIOGRID-ORGANISM-{VERSION}.tab2.zip"
BioGRID_URL = f"https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-{VERSION}"
BioGRID_FILEPATH = f"{RAW_DATA_DIRECTORY}/{BioGRID_FILENAME}" 

In [6]:
# Download
r = requests.get(f"{BioGRID_URL}/{BioGRID_FILENAME}", allow_redirects=True)
with open(BioGRID_FILEPATH, 'wb') as f:
    f.write(r.content)

### select yeast

In [7]:
# Unzip Saccharomyces cerevisiae file
with ZipFile(BioGRID_FILEPATH, 'r') as z:
    BioGRID_sc_FILENAME, = [name for name in z.namelist() if 'cerevisiae' in name.lower()]
    z.extract(BioGRID_sc_FILENAME, RAW_DATA_DIRECTORY)

### SGD annotations

In [8]:
SGD_FILENAME = "sgd.gaf.gz"
SGD_URL = "http://current.geneontology.org/annotations"
SGD_FILEPATH = f"{RAW_DATA_DIRECTORY}/{SGD_FILENAME}"

In [9]:
# Download
r = requests.get(f"{SGD_URL}/{SGD_FILENAME}", allow_redirects=True)
with open(SGD_FILEPATH, 'wb') as f:
    f.write(r.content)

### GO

In [10]:
GO_FILENAME = "go-basic.obo"
GO_URL = "http://purl.obolibrary.org/obo/go"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

In [11]:
# Download
r = requests.get(f"{GO_URL}/{GO_FILENAME}", allow_redirects=True)
with open(GO_FILEPATH, 'wb') as f:
    f.write(r.content)

## DEG

In [3]:
DEG_FILENAME = "deg-e-15.2.zip"
DEG_URL = f"http://origin.tubic.org/deg/public/static/download/{DEG_FILENAME}"
DEG_FILEPATH = f"{RAW_DATA_DIRECTORY}/{DEG_FILENAME}"

In [4]:
# Download
r = requests.get(f"{DEG_URL}", allow_redirects=True)
with open(DEG_FILEPATH, 'wb') as f:
    f.write(r.content)

### select annotations

In [5]:
# Unzip Saccharomyces cerevisiae file
with ZipFile(DEG_FILEPATH, 'r') as z:
    DEG_ANN_FILENAME, = ['degannotation-e.dat']
    z.extract(DEG_ANN_FILENAME, RAW_DATA_DIRECTORY)

# BioGRID PPI

### Define interactor universes

In [4]:
VERSION = "3.5.178"

# load BioGRID file as dataframe
organism_FILENAME = f"BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-{VERSION}.tab2.txt"
organism_FILEPATH = f"{RAW_DATA_DIRECTORY}/{organism_FILENAME}"

# remove entrez ids and scores because of mixed datatypes 
BioGRID_df = pd.read_csv(organism_FILEPATH, delimiter='\t',
                           usecols=[index for index in range(24) if index not in {1,2,18}])

In [5]:
# BioGRID universe is the collection of every gene known to BioGRID
universe = set(BioGRID_df['BioGRID ID Interactor A']) | \
           set(BioGRID_df['BioGRID ID Interactor B'])

systematic_universe = set(BioGRID_df['Systematic Name Interactor A']) | \
                      set(BioGRID_df['Systematic Name Interactor B'])
systematic_universe.remove('-')

In [6]:
# Filter for (reliable) physical interactions
EXPERIMENTAL_SYSTEM = {'Two-hybrid', 
                       'Affinity Capture-Luminescence',
                       'Affinity Capture-MS', 
                       'Affinity Capture-RNA', 
                       'Affinity Capture-Western'}
EXPERIMENTAL_SYSTEM_TYPE = {'physical'} # redundant because of experimental evidence filtering

physical_interaction_df = BioGRID_df[BioGRID_df['Experimental System'].isin(EXPERIMENTAL_SYSTEM)]
physical_interaction_df = physical_interaction_df[
    physical_interaction_df['Experimental System Type'].isin(EXPERIMENTAL_SYSTEM_TYPE)] # just in case...


physical_universe = set(physical_interaction_df['BioGRID ID Interactor A']) | \
                    set(physical_interaction_df['BioGRID ID Interactor B'])

systematic_physical_universe = set(physical_interaction_df['Systematic Name Interactor A']) | \
                                       set(physical_interaction_df['Systematic Name Interactor B'])
systematic_physical_universe.remove('-')

### Define PPI

In [7]:
# PPI in dataframe
PPI_df = physical_interaction_df[
    physical_interaction_df["Systematic Name Interactor A"].str.startswith('Y') & \
    physical_interaction_df["Systematic Name Interactor B"].str.startswith('Y')]

PPI_universe = set(PPI_df['Systematic Name Interactor A']) | \
               set(PPI_df['Systematic Name Interactor B'])

In [8]:
# Reduce PPI to simple network
PPI_nx = nx.from_pandas_edgelist(PPI_df,'Systematic Name Interactor A', 'Systematic Name Interactor B')
PPI_nx.remove_edges_from(nx.selfloop_edges(PPI_nx))

### Summary

In [9]:
print("BioGRID universe sizes:")
print("=========================")
print(f"    -         -    : {len(universe)}")
print(f"systematic    -    : {len(systematic_universe)}")
print(f"    -      physical: {len(physical_universe)}")
print(f"systematic physical: {len(systematic_physical_universe)}")
print("-------------------------")
print(f"PPI population size: {PPI_nx.number_of_nodes()}")

BioGRID universe sizes:
    -         -    : 7175
systematic    -    : 6538
    -      physical: 6553
systematic physical: 6123
-------------------------
PPI population size: 5726


### Save

In [10]:
if not os.path.exists(NETWORK_DIRECTORY):
    os.makedirs(NETWORK_DIRECTORY)

nx.write_edgelist(PPI_nx, f"{NETWORK_DIRECTORY}/PPI_BioGRID.txt", data=False)

# BioGRID PPI (official)

In [6]:
VERSION = "3.5.178"

# load BioGRID file as dataframe
organism_FILENAME = f"BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-{VERSION}.tab2.txt"
organism_FILEPATH = f"{RAW_DATA_DIRECTORY}/{organism_FILENAME}"

# remove entrez ids and scores because of mixed datatypes 
BioGRID_df = pd.read_csv(organism_FILEPATH, delimiter='\t',
                           usecols=[index for index in range(24) if index not in {1,2,18}])

In [7]:
# BioGRID universe is the collection of every gene known to BioGRID
universe = set(BioGRID_df['BioGRID ID Interactor A']) | \
           set(BioGRID_df['BioGRID ID Interactor B'])

official_universe = set(BioGRID_df['Official Symbol Interactor A']) | \
                    set(BioGRID_df['Official Symbol Interactor B'])

In [8]:
# Filter for (reliable) physical interactions
EXPERIMENTAL_SYSTEM = {'Two-hybrid', 
                       'Affinity Capture-Luminescence',
                       'Affinity Capture-MS', 
                       'Affinity Capture-RNA', 
                       'Affinity Capture-Western'}
EXPERIMENTAL_SYSTEM_TYPE = {'physical'} # redundant because of experimental evidence filtering

physical_interaction_df = BioGRID_df[BioGRID_df['Experimental System'].isin(EXPERIMENTAL_SYSTEM)]
physical_interaction_df = physical_interaction_df[
    physical_interaction_df['Experimental System Type'].isin(EXPERIMENTAL_SYSTEM_TYPE)] # just in case...


physical_universe = set(physical_interaction_df['BioGRID ID Interactor A']) | \
                    set(physical_interaction_df['BioGRID ID Interactor B'])

official_physical_universe = set(physical_interaction_df['Official Symbol Interactor A']) | \
                             set(physical_interaction_df['Official Symbol Interactor B'])

### Define PPI

In [9]:
# PPI in dataframe
PPI_df = physical_interaction_df

PPI_universe = set(PPI_df['Official Symbol Interactor A']) | \
               set(PPI_df['Official Symbol Interactor B'])

In [10]:
# Reduce PPI to simple network
PPI_nx = nx.from_pandas_edgelist(PPI_df,'Official Symbol Interactor A', 'Official Symbol Interactor B')
PPI_nx.remove_edges_from(nx.selfloop_edges(PPI_nx))

### Summary

In [11]:
print("BioGRID universe sizes:")
print("=========================")
print(f"    -       -    : {len(universe)}")
print(f"official    -    : {len(official_universe)}")
print(f"    -    physical: {len(physical_universe)}")
print(f"official physical: {len(official_physical_universe)}")
print("-------------------------")
print(f"PPI population size: {PPI_nx.number_of_nodes()}")

BioGRID universe sizes:
    -       -    : 7175
official    -    : 7091
    -    physical: 6553
official physical: 6502
-------------------------
PPI population size: 6502


### Save

In [12]:
if not os.path.exists(NETWORK_DIRECTORY):
    os.makedirs(NETWORK_DIRECTORY)

nx.write_edgelist(PPI_nx, f"{NETWORK_DIRECTORY}/PPI_BioGRID_official.txt", data=False)

# Annotations

## SGD (S. cerevisiae)

In [11]:
SGD_FILENAME = "sgd.gaf.gz"
SGD_FILEPATH = f"{RAW_DATA_DIRECTORY}/{SGD_FILENAME}"

# load SGD gaf-file as dataframe 
with gzip.open(SGD_FILEPATH, 'rt') as gz:
    SGD_df = pd.DataFrame(annotation for annotation in GOA.gafiterator(gz))
    
# Define column with systematic gene names. In SGD this is always the first synonym.
SGD_df['Systematic_ID']  = SGD_df.Synonym.apply(lambda list_:list_[0])

In [12]:
# Filter for proteins
lc_protein_gaf_df = SGD_df[SGD_df['Systematic_ID'].str.startswith('Y')]

# Filter through evidence code
protein_gaf_df = lc_protein_gaf_df[lc_protein_gaf_df['Evidence'].isin(['EXP', 'IDA', 'IPI', 'IMP', 'IGI', 'IEP'])]

# Split into the three GOs
protein_BP_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='P']
protein_MF_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='F']
protein_CC_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='C']

In [13]:
# Get rid of all unnecesarry columns in the GAFs
high_IC_annotations_df = protein_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()

# Split into the three GOs
high_IC_BP_annotations_df = protein_BP_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()
high_IC_MF_annotations_df = protein_MF_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()
high_IC_CC_annotations_df = protein_CC_gaf_df[['Systematic_ID', 'GO_ID']].dropna().drop_duplicates()

### GO DAG extention (Yeast2GO)

In [14]:
# Load obo and gaf files
GO_FILENAME = "go-basic.obo"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

# Create annotations with all upstream terms in dict
go_dag = obo_parser.GODag(GO_FILEPATH)
go2parents = {go_id:{*go_dag[go_id].get_all_parents(), go_id} for go_id in go_dag.keys()}

/home/clusterduck123/Desktop/git/supplements/data/raw-data/go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms


In [15]:
# Create full list/dataframe of annotations
all_annotations_list = [(gene_id, go_term, go_dag[go_term].level) 
                                for _, (gene_id, go_id) in high_IC_annotations_df.iterrows()
                                    for go_term in go2parents[go_id]]
all_annotations_df = pd.DataFrame(
                            all_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()

# Split into the three GOs
all_BP_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "biological_process"]
all_MF_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "molecular_function"]
all_CC_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "cellular_component"]

all_BP_annotations_df = pd.DataFrame(
                            all_BP_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()
all_MF_annotations_df = pd.DataFrame(
                            all_MF_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()
all_CC_annotations_df = pd.DataFrame(
                            all_CC_annotations_list,  
                            columns = ['Systematic_ID', 'GO_ID', 'Level']).drop_duplicates()

### Summary

In [16]:
print("SGD universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(map(lambda x:x[0],all_BP_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_BP_annotations_list)))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(map(lambda x:x[0],all_MF_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_MF_annotations_list)))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(map(lambda x:x[0],all_CC_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_CC_annotations_list)))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(map(lambda x:x[0],all_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_annotations_list)))} GO-IDs ")

SGD universe sizes:
Biological Process : 4532 genes, 4662 GO-IDs 
Molecular Functions: 3564 genes, 2105 GO-IDs 
Cellular Components: 4013 genes, 904  GO-IDs 
--------------------------------------------
All annotations    : 4967 genes, 7671 GO-IDs 


### Save

In [17]:
if not os.path.exists(ANNOTATION_DIRECTORY):
    os.makedirs(ANNOTATION_DIRECTORY)

all_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_all_SGD.csv", index=False)

all_BP_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_BP_SGD.csv", index=False)
all_MF_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_MF_SGD.csv", index=False)
all_CC_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_CC_SGD.csv", index=False)

## BioGRID $\cap$ SGD

In [18]:
PPI_nx = nx.read_edgelist(f"{NETWORK_DIRECTORY}/PPI_BioGRID.txt")

all_annotations_df = pd.read_csv(f"{ANNOTATION_DIRECTORY}/GO_all_SGD.csv")

all_BP_annotations_df = pd.read_csv(f"{ANNOTATION_DIRECTORY}/GO_BP_SGD.csv")
all_MF_annotations_df = pd.read_csv(f"{ANNOTATION_DIRECTORY}/GO_MF_SGD.csv")
all_CC_annotations_df = pd.read_csv(f"{ANNOTATION_DIRECTORY}/GO_CC_SGD.csv")

In [19]:
PPI_annotations_df = all_annotations_df[all_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]

PPI_BP_annotations_df = all_BP_annotations_df[all_BP_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]
PPI_MF_annotations_df = all_MF_annotations_df[all_MF_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]
PPI_CC_annotations_df = all_CC_annotations_df[all_CC_annotations_df.Systematic_ID.isin(PPI_nx.nodes)]

### Summary

In [20]:
print(r"SGD ∩ BioGRID universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(PPI_BP_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_BP_annotations_df.GO_ID))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(PPI_MF_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_MF_annotations_df.GO_ID))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(PPI_CC_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_CC_annotations_df.GO_ID))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(PPI_annotations_df.Systematic_ID))} genes, "
    f"{len(set(PPI_annotations_df.GO_ID))} GO-IDs ")

SGD ∩ BioGRID universe sizes:
Biological Process : 4496 genes, 4658 GO-IDs 
Molecular Functions: 3537 genes, 2097 GO-IDs 
Cellular Components: 3993 genes, 904  GO-IDs 
--------------------------------------------
All annotations    : 4923 genes, 7659 GO-IDs 


### Save

In [21]:
PPI_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_all_BioGRID-SGD.csv", index=False)

PPI_BP_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_BP_BioGRID-SGD.csv", index=False)
PPI_MF_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_MF_BioGRID-SGD.csv", index=False)
PPI_CC_annotations_df.to_csv(f"{ANNOTATION_DIRECTORY}/GO_CC_BioGRID-SGD.csv", index=False)